In [66]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, r2_score
import os
import joblib
import json
from datetime import datetime

In [86]:
# Data Fetching
def fetch_data(ticker: str, start: str = None, end: str = None) -> pd.DataFrame:

    if start is None:
        start = (datetime.today() - pd.DateOffset(years = 2)).strftime("%Y-%m-%d")
    if end is None:
        end = datetime.today().strftime("%Y-%m-%d")
        
    data = yf.download(ticker, start=start, end=end, group_by="column", auto_adjust = True)
    # Flatten MultiIndex columns if necessary
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [col[0] for col in data.columns]
    return data

In [88]:
def compute_features(data: pd.DataFrame) -> pd.DataFrame:
    data = data.copy()
    
    # Basic Returns
    data["Return"] = data["Close"].pct_change()

    for lag in [1, 2, 3, 5]:
        data[f"Return_Lag_{lag}"] = data["Return"].shift(lag)

    # RSI (Wilder's smoothing)
    delta = data["Close"].diff()

    data["gain"] = np.where(delta > 0, delta, 0)
    data["loss"] = np.where(delta < 0, -delta, 0)

    data["avg_gain"] = data["gain"].ewm(alpha=1/14, adjust=False).mean()
    data["avg_loss"] = data["loss"].ewm(alpha=1/14, adjust=False).mean()

    rs = data["avg_gain"] / (data["avg_loss"] + 1e-10)
    data["RSI"] = 100 - (100 / (1 + rs))

    data.drop(columns=["gain", "loss", "avg_gain", "avg_loss"], inplace=True)

    # MACD
    ema12 = data["Close"].ewm(span=12, adjust=False).mean()
    ema26 = data["Close"].ewm(span=26, adjust=False).mean()

    data["MACD"] = ema12 - ema26
    data["MACD_Signal"] = data["MACD"].ewm(span=9, adjust=False).mean()

    # Bollinger Bands

    sma20 = data["Close"].rolling(window=20).mean()
    std20 = data["Close"].rolling(window=20).std()

    data["BB_Upper"] = sma20 + 2 * std20
    data["BB_Lower"] = sma20 - 2 * std20
    data["BB_Width"] = data["BB_Upper"] - data["BB_Lower"]

    # Rolling & Historical Volatility
    data["RollingVolatility"] = data["Return"].rolling(window=20).std()

    for window in [10, 20, 30]:
        data[f"HV_{window}"] = data["Return"].rolling(window).std()

    # ATR (Wilder optional)
    data["High-Low"] = data["High"] - data["Low"]
    data["High-Close"] = np.abs(data["High"] - data["Close"].shift())
    data["Low-Close"] = np.abs(data["Low"] - data["Close"].shift())

    data["TR"] = data[["High-Low", "High-Close", "Low-Close"]].max(axis=1)
    data["ATR"] = data["TR"].ewm(alpha=1/14, adjust=False).mean()

    data.drop(columns=["High-Low", "High-Close", "Low-Close", "TR"], inplace=True)

    # Momentum
    data["RollingMean"] = data["Return"].rolling(window=10).mean()

    # Target Volatility
    data["TargetVolatility"] = data["RollingVolatility"].shift(-1)

    # Drop Warmup + Last Row
    data.dropna(inplace=True)

    return data

In [90]:
features = ["RollingVolatility", "ATR", "RollingMean", "Return",
    "RSI", "MACD", "MACD_Signal",
    "BB_Width",
    "Return_Lag_1", "Return_Lag_2", "Return_Lag_3", "Return_Lag_5",
    "HV_10", "HV_20", "HV_30"]

# Prepare Features & Target
def prepare_X_y(data: pd.DataFrame, feature_list):
    X = data[feature_list]
    y = data["TargetVolatility"]
    return X, y

In [92]:
# Train Model with GridSearchCV & TimeSeriesSplit
def train_model(X_train, y_train):
    param_grid = {
        "n_estimators": [100, 200, 300],
        "max_depth": [None, 5, 10, 20],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
    }
    
    rf = RandomForestRegressor(random_state=123)
    tscv = TimeSeriesSplit(n_splits=3)
    
    grid_search = GridSearchCV(
        estimator=rf,
        param_grid=param_grid,
        cv=tscv,
        n_jobs=-1,
        scoring='neg_mean_absolute_error',
        verbose=1
    )
    
    grid_search.fit(X_train, y_train)
    best_rf = grid_search.best_estimator_
    
    print("Best parameters:", grid_search.best_params_)
    return best_rf

In [94]:
# Evaluate
def evaluate_model_metrics(model, X_test, y_test):
    """
    Returns evaluation metrics for the model on test data.
    """
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Mean Absolute Error: {mae:.6f}")
    print(f"R² Score: {r2:.4f}")
    
    return y_pred, mae, r2

def plot_predictions(y_test, y_pred, data_index, ticker="TICKER"):
    """
    Plots actual vs predicted volatility.
    """
    plt.figure(figsize=(12,5))
    plt.plot(data_index[-len(y_test):], y_test, label="Actual", linewidth=2)
    plt.plot(data_index[-len(y_test):], y_pred, label="Predicted", linewidth=2)
    plt.legend()
    plt.title(f"{ticker} Volatility Prediction (Time-Series GridSearchCV)")
    plt.show()

In [96]:
def get_model_paths(ticker: str, model_dir: str = "models"):
    """
    Returns the directory, model path, and metadata path for a given ticker.
    """
    ticker_dir = os.path.join(model_dir, ticker)
    model_path = os.path.join(ticker_dir, "model.pkl")
    metadata_path = os.path.join(ticker_dir, "metadata.json")
    return ticker_dir, model_path, metadata_path


def save_model_and_metadata(ticker: str, model, model_dir: str = "models"):
    """
    Saves the model and corresponding metadata (ticker, trained_on, version, features).
    """
    ticker_dir, model_path, metadata_path = get_model_paths(ticker, model_dir)
    os.makedirs(ticker_dir, exist_ok=True)
    
    # Save model
    joblib.dump(model, model_path)
    
    # Save metadata
    metadata = {
        "ticker": ticker,
        "trained_on": datetime.now().isoformat(),
        "model_version": "v1.0",
        "features": features
    }
    
    with open(metadata_path, "w") as f:
        json.dump(metadata, f, indent=4)
    
    print(f"Saved model to {model_path}")
    print(f"Saved metadata to {metadata_path}")


def load_model_and_metadata(ticker: str, model_dir: str = "models"):
    """
    Loads the model and metadata if they exist, otherwise returns (None, None).
    """
    ticker_dir, model_path, metadata_path = get_model_paths(ticker, model_dir)
    
    if not os.path.exists(model_path) or not os.path.exists(metadata_path):
        return None, None
    
    model = joblib.load(model_path)
    with open(metadata_path, "r") as f:
        metadata = json.load(f)
    
    return model, metadata

In [98]:
def is_model_stale(metadata: dict, max_age_days: int = 30) -> bool:
    """
    Returns True if model is older than max_age_days.
    """
    trained_on_str = metadata.get("trained_on")
    if trained_on_str is None:
        return True  # no trained date = treat as stale
    
    trained_on = datetime.fromisoformat(trained_on_str).date()
    today = datetime.today().date()
    age = (today - trained_on).days
    
    return age > max_age_days

In [100]:
# Full Pipeline
def run_pipeline(ticker: str, feature_list, plot: bool = False, model_dir: str = "models"):
    """
    Runs the full volatility prediction pipeline for a given ticker.
    
    - If a saved, non-stale model exists, loads the model.
    - Otherwise, trains a new model, saves it + metadata, and uses it.
    
    Returns:
        model, X_test, y_test, y_pred
    """
    # Fetch and process data
    data = fetch_data(ticker)
    data = compute_features(data)
    X, y = prepare_X_y(data, features)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Try to load existing model + metadata
    model, metadata = load_model_and_metadata(ticker, model_dir=model_dir)
    
    if model is not None and metadata is not None and not is_model_stale(metadata):
        print(
            f"Loaded model v{metadata['model_version']} "
            f"for {ticker} (trained_on={metadata['trained_on']})"
        )
    else:
        if model is None:
            print(f"No saved model for {ticker}. Training new model...")
        elif metadata is None:
            print(f"No metadata for {ticker}. Retraining model...")
        else:
            print(f"Model for {ticker} is stale. Retraining... (trained_on={metadata.get('trained_on')})")
        
        model = train_model(X_train, y_train)
        save_model_and_metadata(ticker, model, model_dir=model_dir)
    
    # Evaluate metrics
    y_pred, mae, r2 = evaluate_model_metrics(model, X_test, y_test)
    
    # Optional plot
    if plot:
        plot_predictions(y_test, y_pred, data.index, ticker)
    
    return model, X_test, y_test, y_pred, data

In [113]:
from arch import arch_model
tickers = ["AAPL", "TSLA", "SPY"]

# Store results for later visualization or analysis
results = {}

for ticker in tickers:
    print(f"\n=== Processing {ticker} ===")

    # Run ML model
    model, X_test, y_test, y_pred, data = run_pipeline(ticker, feature_list = features, plot=False)

    # Store RF metrics correctly
    rf_mae = mean_absolute_error(y_test, y_pred)
    rf_r2 = r2_score(y_test, y_pred)

    # Prepare returns for GARCH
    garch_returns = data["Return"].dropna() * 100
    
    # Fit GARCH
    garch = arch_model(garch_returns, vol="Garch", p=1, q=1)
    garch_fit = garch.fit(disp="off")

    # Forecast GARCH volatility for test horizon
    garch_forecast = garch_fit.forecast(horizon=len(y_test))
    garch_vol = garch_forecast.variance.values[-1, :] ** 0.5 / 100

    # Compute GARCH MAE
    garch_mae = mean_absolute_error(y_test, garch_vol)

    print(f"{ticker} -> RF MAE: {rf_mae:.6f}, GARCH MAE: {garch_mae:.6f}")

[*********************100%***********************]  1 of 1 completed


=== Processing AAPL ===
Loaded model vv1.0 for AAPL (trained_on=2025-11-29T15:16:00.553872)
Mean Absolute Error: 0.000693
R² Score: 0.9054
AAPL -> RF MAE: 0.000693, GARCH MAE: 0.003339

=== Processing TSLA ===



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Loaded model vv1.0 for TSLA (trained_on=2025-11-29T15:16:20.977134)
Mean Absolute Error: 0.001593
R² Score: 0.7254
TSLA -> RF MAE: 0.001593, GARCH MAE: 0.010086

=== Processing SPY ===
Loaded model vv1.0 for SPY (trained_on=2025-11-29T15:16:42.001021)
Mean Absolute Error: 0.000382
R² Score: 0.8992
SPY -> RF MAE: 0.000382, GARCH MAE: 0.003238
